<a href="https://colab.research.google.com/github/manasanagendran/goto_ai_projects/blob/main/RAG_Leave_policy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --upgrade langchain

In [4]:
!pip install langchain-community

In [5]:
! pip install pypdf

In [6]:
! pip install openai
! pip install tiktoken
!pip install langchain-community langchain-core


In [7]:
!pip install config

In [8]:
import config
import openai
from langchain_community.embeddings import OpenAIEmbeddings
import tiktoken

In [9]:
embedding = OpenAIEmbeddings(api_key=config.API_KEY)

<ipython-input-9-2102901832>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(api_key=config.API_KEY)


In [10]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/India-Leaves and Holiday Policy-042624.pdf")
pages = loader.load()

In [23]:
pages

[Document(metadata={'producer': 'Acrobat PDFMaker 15 for Word', 'creator': 'Word', 'creationdate': '2018-09-14T16:05:57+00:00', 'author': 'Alison MacDonald', 'keywords': '', 'moddate': '2024-04-25T13:18:25+05:30', 'subject': '', 'title': 'India Leaves and Holiday Policy', 'source': '/content/India-Leaves and Holiday Policy-042624.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='Leave & Holidays Program – India \nIntroduction \nThis program document may be modified, revised, discontinued or amended at any time, in whole or in part, \nfor any reason and without prior notice, consent or approval within the sole discretion of Adobe-India. This \nprogram document contains proprietary and confidential information. It is solely for use by and distribution to \nthe individuals who are subject to this program. \nThe information provided in this publication is for information purposes only and does not constitute a \ncontract of employment. \nEligibility \nTo apply for a leav

In [29]:
all_text = " ".join([doc.page_content for doc in pages])

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [31]:
r_splitter = RecursiveCharacterTextSplitter(
     separators=['\n\n', '\n', '.' ],
     chunk_size = 200,
    chunk_overlap  = 0,
)
chunks = r_splitter.split_text(all_text)
print("Chunks: ", chunks)
print("Length of chunks: ", len(chunks))

Chunks:  ['Leave & Holidays Program – India \nIntroduction \nThis program document may be modified, revised, discontinued or amended at any time, in whole or in part,', 'for any reason and without prior notice, consent or approval within the sole discretion of Adobe-India. This', 'program document contains proprietary and confidential information. It is solely for use by and distribution to \nthe individuals who are subject to this program.', 'The information provided in this publication is for information purposes only and does not constitute a \ncontract of employment. \nEligibility', 'To apply for a leave of absence, you must be a salaried employee. Interns, temporary and seasonal employees,', 'individuals who perform services as a contractor to Adobe-India are not eligible to participate. \n1. National / Festival holidays', 'In keeping with the National and Festival Holidays Act, India employees will be paid 10 days per calendar', 'year as holidays. This will be as-per a pre-publis

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [39]:
def retrieve_with_tfidf(chunks, query, top_k=5):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(chunks + [query])

    query_vec = tfidf_matrix[-1]
    doc_vecs = tfidf_matrix[:-1]

    scores = cosine_similarity(query_vec, doc_vecs).flatten()
    top_indices = scores.argsort()[-top_k:][::-1]

    return [(chunks[i], scores[i]) for i in top_indices]

In [43]:
def retrieve_score_results(query):
  results = retrieve_with_tfidf(chunks, query)
  for i, (chunk, score) in enumerate(results):
    print(f"\nTop {i+1} Score: {score:.2f}:\n{chunk}")

In [45]:
query_1 = "How many holidays per year?"
retrieve_score_results(query_1)


Top 1 Score: 0.28:
year as holidays. This will be as-per a pre-published holiday calendar. For further information, employees

Top 2 Score: 0.20:
Adobe will provide India employees with 8 days of paid casual time off, per calendar year. Any unused casual

Top 3 Score: 0.20:
In keeping with the National and Festival Holidays Act, India employees will be paid 10 days per calendar

Top 4 Score: 0.19:
Adobe will provide India employees up to 10 days of paid sick time per calendar year. Employees availing sick

Top 5 Score: 0.17:
year). Any unused vacation leaves will carry over to the next calendar year up to a maximum of 45 days. Once it


In [47]:
query_2 = "Give details about sick leave?"
retrieve_score_results(query_2)


Top 1 Score: 0.21:
also would need to give his/her contact details to the Manager so that Manager can contact the employee 
during the absence in case of an emergency. Employees can apply for leaves through Work Day

Top 2 Score: 0.18:
Adobe will provide India employees up to 10 days of paid sick time per calendar year. Employees availing sick

Top 3 Score: 0.16:
The Company provides the following leave for all regular employees. 
Vacation Leave 
Casual Leave 
Sick Leave 
Company Break Period 
3. Vacation Leave

Top 4 Score: 0.14:
leave will lapse at the end of the calendar year. Employees can apply for leaves through Work Day Workday 
5. Sick Leave

Top 5 Score: 0.10:
Workday 
SABBATICAL PLAN 
Please read details on the Sabbatical Plan through this link: 
https://benefits.adobe.com/in/time-off/sabbatical 
LEAVE OF ABSENCE (Medical LOA, Personal LOA)
